<a href="https://colab.research.google.com/github/psych0man/Auto-Completion-using-N-Gram-Models/blob/master/N_gram_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Modules


In [ ]:
import math
import random
import numpy as np
import pandas as pd
import nltk

In [ ]:
nltk.data.path.append('.')

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Pre-Processing 

**Reading the Text**

In [ ]:
with open("text.txt", "r") as f:
  data = f.read()

**Converting corpus into sentences**

In [ ]:
def obtain_sentences(data):
  sentences = data.split("\n")
  sentences = [s.strip() for s in sentences]
  sentences = [s for s in sentences if len(s) > 0]
  return sentences

**Tokenizing the Sentences**

In [ ]:
def tokenize_the_corpus(sentences):
  tokenized_sentences = []
  for sentence in sentences:
    sentence = sentence.lower()
    tokenized_sentence = nltk.word_tokenize(sentence)
    tokenized_sentences.append(tokenized_sentence)
  return tokenized_sentences

In [ ]:
sentences = obtain_sentences(data)
tokenized_sentences = tokenize_the_corpus(sentences)

**Splitting the Corpus**

In [ ]:
train_size = int(len(tokenized_sentences) * 0.8)
train_data = tokenized_sentences[0:train_size]
test_data = tokenized_sentences[train_size:]

Instead of using every word of the corpus we generally use the words which occur most frequently 

In [ ]:
def count_the_words(sentences):

  word_counts = {}
  for sentence in sentences:
    for token in sentence:
      if token not in word_counts.keys():
        word_counts[token] = 1
      else:
        word_counts[token] += 1
  return word_counts


**Handling OOV Words**

In [ ]:
def handling_oov(tokenized_sentences, count_threshold):
  closed_vocabulary = []
  words_count = count_the_words(tokenized_sentences)
  for word, count in words_count.items():
    if count >= count_threshold :
      closed_vocabulary.append(word)

  return closed_vocabulary

**Converting OOV to UNK Tokens**

In [ ]:
def unk_tokenize(tokenized_sentences, vocabulary, unknown_token = "<unk>"):
  vocabulary = set(vocabulary)
  new_tokenized_sentences = []
  for sentence in tokenized_sentences:
    new_sentence = []
    for token in sentence:
      if token in vocabulary:
        new_sentence.append(token)
      else:
        new_sentence.append(unknown_token)
    new_tokenized_sentences.append(new_sentence)
  return new_tokenized_sentences

**Final Steps**

In [ ]:
def preprocess(train_data, test_data, count_threshold):
  vocabulary = handling_oov(train_data, count_threshold)
  new_train_data = unk_tokenize(train_data, vocabulary)
  new_test_data = unk_tokenize(test_data, vocabulary)

  return new_train_data, new_test_data, vocabulary 

In [ ]:
min_freq = 2
final_train, final_test, vocabulary = preprocess(train_data, test_data, min_freq)

# N-Gram Model

In [ ]:
def count_n_grams(data, n, start_token = "<s>", end_token = "<e>"):
  n_grams = {}
  for sentence in data:
    sentence = [start_token]*n + sentence + [end_token]
    sentence = tuple(sentence)

    m = len(sentence) if n==1 else len(sentence)-1
    for i in range(m):
      n_gram = sentence[i:i+n]
      if n_gram in n_grams.keys():
        n_grams[n_gram] += 1
      else:
        n_grams[n_gram] = 1
  return n_grams

In [ ]:
def prob_for_single_word(word, previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary_size, k = 1.0):
  previous_n_gram = tuple(previous_n_gram)
  previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
  denom = previous_n_gram_count + k * vocabulary_size
  nplus1_gram = previous_n_gram + (word,)
  nplus1_gram_count = nplus1_gram_counts[nplus1_gram] if nplus1_gram in nplus1_gram_counts else 0
  num = nplus1_gram_count + k
  prob = num / denom
  return prob

In [ ]:
def probs(previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0):
  previous_n_gram = tuple(previous_n_gram)
  vocabulary = vocabulary + ["<e>", "<unk>"]
  vocabulary_size = len(vocabulary)
  probabilities = {}
  for word in vocabulary:
    probability = prob_for_single_word(word, previous_n_gram, 
                                           n_gram_counts, nplus1_gram_counts, 
                                           vocabulary_size, k=k)
    probabilities[word] = probability

  return probabilities

In [ ]:
def count_matrix(nplus1_gram_counts, vocabulary):
  vocabulary = vocabulary + ["<e>", "<unk>"]
  n_grams = []
  for n_plus1_gram in nplus1_gram_counts.keys():
    n_gram = n_plus1_gram[0:-1]
    n_grams.append(n_gram)
  n_grams = list(set(n_grams))

  row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
  col_index = {word:j for j, word in enumerate(vocabulary)}
    
  nrow = len(n_grams)
  ncol = len(vocabulary)
  count_matrix = np.zeros((nrow, ncol))
  for n_plus1_gram, count in nplus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
    
  count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
  return count_matrix

In [ ]:
def prob_matrix(nplus1_gram_counts, vocabulary, k):
  countmatrix = count_matrix(nplus1_gram_counts, unique_words)
  countmatrix += k
  prob_matrix = countmatrix.div(countmatrix.sum(axis=1), axis=0)
  return prob_matrix

# Evaluation using Perplexity 

In [ ]:
def perplexity(sentence, n_gram_counts, nplus1_gram_counts, vocabulary_size, k=1.0):
    n = len(list(n_gram_counts.keys())[0]) 
    sentence = ["<s>"] * n + sentence + ["<e>"]
    sentence = tuple(sentence)
    
    N = len(sentence)

    product_pi = 1.0

    for t in range(n, N): 

        n_gram = sentence[t-n:t]

        word = sentence[t]

        probability = prob_for_single_word(word,n_gram, n_gram_counts, nplus1_gram_counts, len(unique_words), k=1)

        product_pi *= 1 / probability

    perplexity = product_pi**(1/float(N))
 
    return perplexity

# Putting Everything Together: Auto-Complete System

In [ ]:
def auto_complete(previous_tokens, n_gram_counts, nplus1_gram_counts, vocabulary, k=1.0, start_with=None):
    n = len(list(n_gram_counts.keys())[0]) 
    previous_n_gram = previous_tokens[-n:]
    probabilities = probs(previous_n_gram,n_gram_counts, nplus1_gram_counts,vocabulary, k=k)

    suggestion = None
    max_prob = 0

    for word, prob in probabilities.items():
        if start_with != None: 
            if not word.startswith(start_with):
                continue 

        if prob > max_prob: 

            suggestion = word
            max_prob = prob

    return suggestion, max_prob

**Extending to Multiple Suggestions**

In [ ]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        nplus1_gram_counts = n_gram_counts_list[i+1]
        
        suggestion = auto_complete(previous_tokens, n_gram_counts,
                                    nplus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [ ]:
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(final_train, n)
    n_gram_counts_list.append(n_model_counts)

In [ ]:
previous_tokens = ["i", "am", "to"]
tmp_suggest4 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest4)

The previous words are ['i', 'am', 'to'], the suggestions are:


[('be', 0.02698476197403353),
 ('go', 0.0001342822613132805),
 ('have', 0.00013432735576600176),
 ('how', 6.717270101430779e-05)]